In [ ]:
import gymnasium as gym
import aero_gym
from aero_gym.tools import evaluate, plotfile, animaterender, animaterender_contour
import matplotlib
import math
import numpy as np
from juliacall import Main
import logging

In [ ]:
logging.getLogger().setLevel(logging.INFO)

In [ ]:
matplotlib.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['image.interpolation'] = 'none'

In [ ]:
t_max = 10
delta_t = 0.1
t = np.linspace(0, t_max, int(t_max/delta_t)+1)

In [ ]:
h_ddot_impulse = np.zeros(len(t))
h_ddot_impulse[0] = 1.0
h_ddot_ones = np.ones(len(t))
h_ddot_binary_periodic = (np.sin(t) > 0).astype(int) - 0.01*(np.sin(t) < 0).astype(int)

In [ ]:
mylevels = np.concatenate([[-100], np.linspace(-20, 20, num=30), [100]])

## No h_ddot

In [ ]:
env = gym.make(
    'aero_gym/viscous_flow-v0',
    render_mode="grid",
    t_max=t_max,
    delta_t=delta_t,
    Re=200,
    grid_Re=4,
    observe_vorticity_field=True,
    observe_previous_lift=True,
    observe_previous_pressure=True,
    pressure_sensor_positions=[-0.3,-0.2,-0.1,0.0,0.1,0.2,0.3],
    h_ddot_scale=1,
    alpha_ddot_scale=1,
    vorticity_scale=10.0
)

Test if the `info` gets populated after one time step

In [ ]:
obs, info = env.reset();
info

In [ ]:
obs, _, _, _, info = env.step([0]);
info

In [ ]:
obs, info, render_list = evaluate(env)
print(f"{info['episode']['t'][0]} seconds")

In [ ]:
%config InlineBackend.figure_format = 'svg'
fig, axarr = plotfile(info)

In [ ]:
xg, yg = Main.seval("xg, yg = coordinates(vorticity(integrator),g)")
pivot_idx = (env.unwrapped.a, 0)
anim = animaterender_contour(info, xg, yg, render_list, pivot_idx, mylevels);
anim

## Impulse

In [ ]:
env.reset(options={"h_ddot_prescribed":h_ddot_impulse});

In [ ]:
obs, info, render_list = evaluate(env)

In [ ]:
%config InlineBackend.figure_format = 'svg'
fig, axarr = plotfile(info)
axarr[0,0].plot(info["solver_t_hist"], info["solver_fy_hist"])

In [ ]:
xg, yg = Main.seval("xg, yg = coordinates(vorticity(integrator),g)")
pivot_idx = (env.unwrapped.a, 0)
anim = animaterender_contour(info, xg, yg, render_list, pivot_idx, mylevels);
anim

## Step

In [ ]:
env.reset(options={"h_ddot_prescribed":0.1*h_ddot_ones});

In [ ]:
obs, info, render_list = evaluate(env)

In [ ]:
%config InlineBackend.figure_format = 'svg'
fig, axarr = plotfile(info)
axarr[0,0].plot(info["solver_t_hist"], info["solver_fy_hist"])

In [ ]:
xg, yg = Main.seval("xg, yg = coordinates(vorticity(integrator),g)")
pivot_idx = (env.unwrapped.a, 0)
anim = animaterender_contour(info, xg, yg, render_list, pivot_idx, mylevels);
anim

## Gaussian

In [ ]:
def gaussian(x, a, b, c):
    return a * math.exp(-(x - b) ** 2 / (2 * c ** 2))

def dgaussian(x, a, b, c):
    return a * -(x - b) / c ** 2 * math.exp(-(x - b) ** 2 / (2 * c ** 2))

def ddgaussian(x, a, b, c):
    return a * (x ** 2 + b ** 2 - 2 * b * x - c ** 2) / c ** 4 * math.exp(-(x - b) ** 2 / (2 * c ** 2))

In [ ]:
a = -2/3
b = 3.0
c = 0.4
h_ddot_dgaussian = [0.25 * dgaussian(ti, a, b, c) for ti in t]

a = 1/6
b = 8.0
c = 0.4
alpha_ddot_dgaussian = [ddgaussian(ti, a, b, c) for ti in t]

In [ ]:
env.reset(options={"h_ddot_prescribed":h_ddot_dgaussian});

In [ ]:
obs, info, render_list = evaluate(env, alpha_ddot_prescribed=alpha_ddot_dgaussian)

In [ ]:
%config InlineBackend.figure_format = 'svg'
fig, axarr = plotfile(info)
axarr[0,0].plot(info["solver_t_hist"], info["solver_fy_hist"])

In [ ]:
xg, yg = Main.seval("xg, yg = coordinates(vorticity(integrator),g)")
pivot_idx = (env.unwrapped.a, 0)
matplotlib.rcParams['image.interpolation'] = 'none'
anim = animaterender_contour(info, xg, yg, render_list, pivot_idx, mylevels, animate_every=2);
anim